In [4]:

###########################################################################
# See: ~/.ipython/profile_default/startup/ipython_config.py
###########################################################################

append_path = os.path.abspath('/project/lgrandi/dbarge')

if not append_path in sys.path:
    sys.path.append(append_path)
    
import python_utils.python_utils as pyutils
import keras_utils.keras_utils   as kutils

from helpers import *               # used to check train/test data
from pax_utils import s1s2_utils    # used to get s2 integral columns
from xy_s2integrals_models import * # ML modles

#pyutils.printVersions()
#kutils.printVersions()
print("Ready")


Ready


In [5]:

####################################################################################################
####################################################################################################

nEventsTrain = 100000
file_all = '/project/lgrandi/dbarge/simulation/wimp/pax_v6.8.3/merged/merged_all_200000.pkl'


In [36]:

####################################################################################################
####################################################################################################

model_name_h5 = 'models/'

#model_name_h5 += 'model_xy_s2integrals_dnn_elu_layers-127-127-2_epochs10_acc9848.h5'
#model_name_h5 += 'model_xy_s2integrals_dnn_elu_layers-127-127-2_epochs10_acc9905.h5'
#model_name_h5 += 'model_xy_s2integrals_dnn_elu_layers-127-127-2_epochs10_acc9826.h5'
#model_name_h5 += 'model_xy_s2integrals_dnn_elu_layers-127-508-127-2_epochs10_acc9904.h5'
#model_name_h5 += 'model_xy_s2integrals_dnn_elu_acc9908_epochs10_layers-127-127-127-2'
model_name_h5 += 'model_xy_s2integrals_dnn_elu_acc9920_epochs50_layers-127-127-31-2'

model_name_h5 += '.h5'


file_out      = './predictions/' + os.path.basename(model_name_h5).replace('.h5', '.pkl')


####################################################################################################
####################################################################################################

df_all = pd.read_pickle(file_all)
df_all = df_all[df_all['intr_count'] == 1].reset_index(drop=True)
df_all   = df_all[nEventsTrain:][:]


####################################################################################################
####################################################################################################

df_train_truth = df_all[:][['x', 'y']]
df_train_input = df_all[:][s1s2_utils.getS2integralsDataFrameColumns()]


####################################################################################################
####################################################################################################

checkTrainingData(df_train_input, df_train_truth)




Input shape: (20967, 127)
Truth shape: (20967, 2)

Input DataFrame: 


,s2_area_000,s2_area_001,s2_area_002,s2_area_003,s2_area_004,s2_area_005,s2_area_006,s2_area_007,s2_area_008,s2_area_009,...,s2_area_117,s2_area_118,s2_area_119,s2_area_120,s2_area_121,s2_area_122,s2_area_123,s2_area_124,s2_area_125,s2_area_126
100000,1.707281,0.0,0.0,0.725097,0.000000,0.929653,0.748697,0.000000,0.000000,2.321928,...,0.000000,0.0,3.824396,0.871452,12.733224,70.347543,21.697417,1.314791,1.191852,8.218960
100001,2.517367,0.0,0.0,0.492983,3.377065,0.000000,2.676335,4.193884,3.148234,4.658021,...,2.927112,0.0,0.000000,2.789005,4.198920,19.901871,18.400299,4.964570,6.444708,2.787059



Truth DataFrame:


,x,y
100000,11.452923,-4.799238
100001,19.004622,-14.180182


In [37]:

####################################################################################################
####################################################################################################

model       = load_model(model_name_h5)
arr_xy_pred = model.predict(df_train_input.values)

print("\nLoaded Model: " + model_name_h5)
print("Predicted\n")




Loaded Model: models/model_xy_s2integrals_dnn_elu_acc9920_epochs50_layers-127-127-31-2.h5
Predicted



In [38]:

####################################################################################################
####################################################################################################

arr_x_true = df_train_truth[:]['x'].as_matrix()
arr_y_true = df_train_truth[:]['y'].as_matrix()

arr_x_pred = arr_xy_pred[:, 0]
arr_y_pred = arr_xy_pred[:, 1]

arr_dx     = arr_x_true - arr_x_pred
arr_dy     = arr_y_true - arr_y_pred



In [39]:

####################################################################################################
####################################################################################################

cols_out         = ['event_number', 'x', 'y', 'intr_x', 'intr_y', 'intr_x_nn', 'intr_y_nn']

df_out           = df_all[:][cols_out]
df_out['x_pred'] = arr_x_pred
df_out['y_pred'] = arr_y_pred


df_out.to_pickle(file_out)


####################################################################################################
####################################################################################################

df_test = pd.read_pickle(file_out)

display(df_test[0:5][:])
print()



,event_number,x,y,intr_x,intr_y,intr_x_nn,intr_y_nn,x_pred,y_pred
100000,165461,11.452923,-4.799238,10.598371,-4.364035,10.831001,-4.523721,11.126697,-4.824415
100001,165462,19.004622,-14.180182,16.832707,-14.089599,16.632289,-13.558204,17.363718,-14.861678
100002,165463,-0.713852,-30.766732,-0.623434,-27.057018,-1.042401,-27.421310,-0.527274,-29.145491
100003,165468,-31.193505,-19.178735,-27.057018,-14.338972,-25.929953,-13.704764,-30.931894,-17.428183
100004,165469,15.701762,-12.451200,13.341479,-10.348997,13.449440,-11.024782,14.253854,-11.283692
